# Archery Capstone
This capstone project was created in support of the CodeYou program to showcase the materials covered in the Data Analysis pathway. 
The code blocks below will walk you through the following:<br>
1. Accessing three layers created within ArcGIS Online to collect archery related stats from 3D target events<br>
2. Pulling these layers down to data frames to work with separately from the authoritative data within ArcGIS Online<br>
3. Cleaning and joining the data so it can be used for futher analysis and graphic creation<br>
4. Creating graphics and shot summaries for the archer to reference and analyze for trends

## Data Collection
Data was created and collected using esri's ArcGIS Online platform in support of this project. Three distinct layers will be referenced for this project:<br>
1.  A lookup table was created containing the various 3D target animals along with information on the color, size, name, and target pattern.<br> 
2.  A Survey123 was created to collect shoot conditions for each tournament.  Data was collected to capture the shoot name, shoot location (indoor or outdoor), shoes (minimal, hiking boots, sandals, gym shoes), temperature, weather conditions present (sunny, cloudy, humid, windy, foggy, light rain, heavy rain, stormy), and personal conditions (well rested, tired, sick, sore, hungry, full).<br>
3.  A Survey123 was created to collect several details for each shot on the individual target and appearance (animal, range, position, elevation, lighting), shot execution (confidence, release details, execution details), and placement (score, aim placement, actual arrow impact). 

### Import Modules

This section imports the various modules required for the code to execute. 

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd
import pandasql as psql
import requests
print("Modules Imported")

### Define Variables

This section defines any variables which may be referenced in subsequent code blocks below.

### Download AGOL layers to data frames

This section connects to ArcGIS Online and downloads the three datasets to dataframes for cleaning and analysis.

There is an issue with downloading the animals AGOL layer.  Look at fixing that or pull it to a CSV in the data folder and then send it to a data frame from the CSV

In [50]:
# Connect to ArcGIS Online
gis = GIS()

# Item ID of the public layer
item_id = "f48b43672acf4aa1858fcbffe15e33cc"

# Get the item
item = gis.content.get(item_id)
print(item.type)

# If the item is a Feature Layer, access the first layer in the item
if 'Feature Service' in item.type:
    feature_layer = item.tables
    print(item.tables)

    # Query all features
    features = feature_layer.query()
    print(features)

    # Convert to a Spatial DataFrame
    sdf = features.sdf
    
    # Print the dataframe
    print(sdf)


Feature Service
[<Table url:"https://services6.arcgis.com/MQNEtGZdkDmSDKfq/arcgis/rest/services/AnimalLookup_view/FeatureServer/0">]


AttributeError: 'list' object has no attribute 'query'

In [46]:
# Connect to ArcGIS Online
# gis = GIS("https://sso.maps.arcgis.com/")
gis = GIS()

# Define ItemIDs for AGOL layers
Animals = "990f64ab39b24e5b9aa9febc336b72fa"
Shot = "f0314c505e83428d8ce6894a8d73eb09"
Shoot = "5fa46f5f47e841b1b2454271cd346d8f"

# Search for the layers by title
layer3_Shoot = gis.content.get(Shoot)
print (layer3_Shoot.type)
layer2_Shot = gis.content.get(Shot)
print (layer2_Shot.type)
layer1_Animals = gis.content.get(Animals)
print (layer1_Animals.type)



# Access the first layer in each item
feature_layer2_Shot = layer2_Shot.layers[0]
feature_layer3_Shoot = layer3_Shoot.layers[0]
print (feature_layer2_Shot)
print (feature_layer3_Shoot)
feature_layer1_Animals = layer1_Animals.layers
print (feature_layer1_Animals)


# Query the layers to get all features
# features1 = layer1_Animals.query(where="1=1")
# features2 = layer2_Shot.query(where="1=1")
# features3 = layer3_Shoot.query(where="1=1")
features1 = layer1_Animals.query()
features2 = layer2_Shot.query()
features3 = layer3_Shoot.query()

# Convert features to pandas DataFrames
df1_Animals = features1.sdf
df2_Shot = features2.sdf
df3_Shoot = features3.sdf

# Print the DataFrames
print(df1_Animals.head())
print(df2_Shot.head())
print(df3_Shoot.head())

Feature Service
Feature Service
Feature Service
<FeatureLayer url:"https://services6.arcgis.com/MQNEtGZdkDmSDKfq/arcgis/rest/services/survey123_c0aeabe39091491e937f8938d064f6f8/FeatureServer/0">
<FeatureLayer url:"https://services6.arcgis.com/MQNEtGZdkDmSDKfq/arcgis/rest/services/survey123_4cefed1f94564642baf9db2aeb6e23a3/FeatureServer/0">
[]


AttributeError: 'Item' object has no attribute 'query'

In [ ]:

# Search for the layers by title
layer1_Animals = gis.content.search("AnimalLookup", item_type="Table", max_items=1)[0]
layer2_Shot = gis.content.search("survey", item_type="Feature Layer", max_items=1)[0]
layer3_Shoot = gis.content.search("title:YourLayerTitle3", item_type="Feature Layer", max_items=1)[0]

# # Search for the layers by title
# layer1_Animals = gis.content.get("990f64ab39b24e5b9aa9febc336b72fa")
# layer2_Shot = gis.content.get("f0314c505e83428d8ce6894a8d73eb09")
# layer3_Shoot = gis.content.get("5fa46f5f47e841b1b2454271cd346d8f")

# Access the first layer in each item
feature_layer1_Animals = layer1_Animals.layers[0]
feature_layer2_Shot = layer2_Shot.layers[0]
feature_layer3_Shoot = layer3_Shoot.layers[0]

# Query the layers to get all features
features1 = feature_layer1_Animals.query(where="1=1")
features2 = feature_layer2_Shot.query(where="1=1")
features3 = feature_layer3_Shoot.query(where="1=1")

# Convert features to pandas DataFrames
df1_Animals = features1.sdf
df2_Shot = features2.sdf
df3_Shoot = features3.sdf

# Print the DataFrames
print(df1_Animals.head())
print(df2_Shot.head())
print(df3_Shoot.head())

### Clean data frames using SQL

1. Populate fields<br>
    - Score to Scored Ring<br>
    - Adjustment Made to No if null

In [ ]:
# Example SQL query to select specific columns and filter rows
query = """
SELECT column1, column2
FROM df1
WHERE column3 > 100
"""
result_df = psql.sqldf(query, locals())

# Print the result of the SQL query
print(result_df)

2. Clean data frames using SQL
-  Create flag fields on multi select questions
    -  Adjustment
        -  Increase_Yardage, Decrease_Yardage, Sight_Left, Sight_Right
    -  Position
        -  Quartered_to, Quartered_From, Broadside, Leaning_Forward, Leaning_Back
    -  Shot Placement
        -  On_Aim_Point, High, Low, Left, Right, Miss, Kick_Out
    -  Shot Execution
        -  Unsure, Felt_Good, Bad_Shoulder, Bad_Release, Movement_in_Shot, Unable_to_Center_Bubble, Dip, Push_Up, Glare, other
    -  Release Details
        -  Too_Soon, Fast, Slow, Off_Face, Hand_Moved
    -  Weather Conditions Present
    -  Personal Conditions

5. Clean data frames using SQL
    - Create a new layer by doing a join on t1.Animals (t1.TargetName = t3.target) and t2.shooting conditions (t3.date_and_time = t2.date_and_time_of_Shoot) to t3.shot report out

Create graphics
    Shot placement on color
    Shot placement on aim point
    Shot Placement on range
    Shot placement on 
    Animal Size on color
    Shot placement on position
    Overall Shot Placement
    Release Details on scored ring
    

Create Readme
    Modules Needed
    

Future Work
    1.  Identify further parameters for performing analysis
    2.  Fine tune the shoot report mobile survey to collect additional informaiton (i.e. which direction is the target facing), populate defaults, and improve the data entry process, and allow for open ranges. 
    3.  Create a PDF or Word Report for daily shoot summaries
    
    